<a href="https://colab.research.google.com/github/fatrrr/pantura/blob/main/road_damage_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib as plt
import os
import cv2
import sklearn

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
import zipfile
zip_dir = "/content/drive/Shareddrives/Product Capstone Project/colab_notebook/road_damage_prioritize_dataset.zip"
destination_dir= "/content/drive/Shareddrives/Product Capstone Project/colab_notebook/"

with zipfile.ZipFile(zip_dir, 'r') as zip_ref:
  zip_ref.extractall(destination_dir)

zip_ref.close()
'''

'\nimport zipfile\nzip_dir = "/content/drive/Shareddrives/Product Capstone Project/colab_notebook/road_damage_prioritize_dataset.zip"\ndestination_dir= "/content/drive/Shareddrives/Product Capstone Project/colab_notebook/"\n\nwith zipfile.ZipFile(zip_dir, \'r\') as zip_ref:\n  zip_ref.extractall(destination_dir)\n\nzip_ref.close()\n'

**Transfer Learning**

In [ ]:
'''
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
m.build([None, expect_img_size, expect_img_size, 3])  # Batch input shape.
'''

'\nm = tf.keras.Sequential([\n\xa0 \xa0 hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0   \xa0 \xa0trainable=False), \xa0# Can be True, see below.\n\xa0 \xa0 tf.keras.layers.Dense(num_classes, activation=\'softmax\')\n])\nm.build([None, expect_img_size, expect_img_size, 3]) \xa0# Batch input shape.\n'

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
pixels = 224
FV_SIZE = 1280
IMAGE_SIZE =(pixels, pixels)

feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=False)

In [ ]:
'''NUM_LAYERS = 10

if do_fine_tuning:
    feature_extractor.trainable = True
    
    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False'''

'NUM_LAYERS = 10\n\nif do_fine_tuning:\n    feature_extractor.trainable = True\n    \n    for layer in model.layers[-NUM_LAYERS:]:\n        layer.trainable = True\n\nelse:\n    feature_extractor.trainable = False'

# Image Processing

In [ ]:
data_dir =  "/content/drive/Shareddrives/Product Capstone Project/colab_notebook/sih_road_dataset"
labels = ["tidak_ada_kerusakan", "rendah", "sedang", "tinggi"]
x = []
y = []
for label in labels:
    data = os.path.join(data_dir,label)
    for image in os.listdir(data):
        try:
            img = cv2.imread(os.path.join(data, image), cv2.IMREAD_COLOR)
            img = cv2.resize(img, IMAGE_SIZE)
            
            x.append(img)
            y.append(labels.index(label))
            
        except Exception as e:
            pass

In [ ]:
np.unique(y)
np.unique(x)

x = np.array(x)/255.0
y = np.array(y)

x.shape
y.shape

x = x.reshape(-1, pixels, pixels, 3)
x.shape

y = y.reshape(-1, 1)

In [ ]:
y = tf.keras.utils.to_categorical(y, 4)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagenerator = ImageDataGenerator(
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=False,
      vertical_flip=False,
      fill_mode='nearest')

datagenerator.fit(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
train_generator = datagenerator.flow(x_train, y_train, batch_size=32)
val_generator = datagenerator.flow(x_test, y_test)

# Model

**Model**

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(4, activation='softmax'),
])

#Load Weight

model.load_weights('/content/drive/Shareddrives/Product Capstone Project/colab_notebook/checkpoints/')

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              4049564   
                                                                 
 dense_1 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 4,054,688
Trainable params: 5,124
Non-trainable params: 4,049,564
_________________________________________________________________


In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=4,restore_best_weights=True)

model.fit(
    train_generator,
    epochs=1,
    callbacks=[earlystop],
    validation_data=val_generator
)

49/49 [==============================] - 232s 5s/step - loss: 0.0340 - accuracy: 0.9884 - val_loss: 0.0406 - val_accuracy: 0.9846


In [ ]:
#Save weight

#weight_path_ckpt = "/content/drive/Shareddrives/Product Capstone Project/colab_notebook/checkpoints/"
#model.save_weights(weight_path_ckpt)

In [ ]:
#tf.keras.backend.clear_session()

In [ ]:
#tensorboard
'''import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    train_generator,
    epochs=10,
    callbacks=[tensorboard_callback],
    validation_data=val_generator
)'''

In [ ]:
#!tensorboard dev upload --logdir ./logs